In [54]:
import pandas as pd
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test_data.csv')

In [55]:
print("训练集探索")
print(df_train.head())

训练集探索
   id         1         2         3         4         5         6         7  \
0   1  0.563650  1.069229 -0.837759 -1.122021  0.433296  0.770755 -0.477153   
1   2  0.061333  0.058830  0.056952  0.068634  0.073433  0.072390  0.042975   
2   3  0.035736  0.010964 -0.164872 -0.167714 -0.125075 -0.104771 -0.016650   
3   4 -0.046700  0.060913  0.009340 -0.093400 -0.067817  0.022335  0.006091   
4   5  0.162922 -0.377662  0.014457  0.565437 -0.203369 -0.511508  0.410961   

          8         9  ...      5992      5993      5994      5995      5996  \
0 -0.588421  0.455224  ... -0.050761  0.220506  0.036548 -0.097461 -0.084060   
1 -0.007302 -0.026286  ...  0.061333  0.107437  0.104516  0.063419 -0.014394   
2  0.151471  0.137258  ...  4.272044 -1.991455 -2.922208  1.937039  0.704156   
3 -0.076751 -0.032893  ...  0.095025 -0.000406  0.091776  0.074314 -0.082842   
4  0.228546 -0.515244  ... -0.093563 -0.263632  0.114517  0.209541 -0.184851   

       5997      5998      5999      6

In [6]:
# 提取训练集的y集
y_train = pd.DataFrame()
y_train['label'] = df_train['label']
df_train.drop(columns='label', inplace=True)
y_train

In [2]:
# 定义函数计算时序数据特征值，参考tsfresh，但是减少了特征量
import numpy as np
from scipy import stats
def get_t(df):
    mean_list = []
    std_list = []
    var_list = []
    min_list = []
    max_list = []
    median_list = []
    skew_list = []
    kuri_list = []
    x = pd.DataFrame()
    for i in range(len(df)):
        mean_list.append(np.mean(df.iloc[i][1:].values))
        std_list.append(np.std(df.iloc[i][1:].values))
        var_list.append(np.var(df.iloc[i][1:].values))
        min_list.append(np.min(df.iloc[i][1:].values))
        max_list.append(np.max(df.iloc[i][1:].values))
        median_list.append(np.median(df.iloc[i][1:].values))
        skew_list.append(stats.skew(df.iloc[i][1:].values))
        kuri_list.append(stats.kurtosis(df.iloc[i][1:].values))
    x['mean'] = mean_list
    x['std'] = std_list
    x['var'] = var_list
    x['min'] = min_list
    x['max'] = max_list
    x['median'] = median_list
    x['skew'] = skew_list
    x['kuri'] = kuri_list
    return x

In [7]:
# 对训练集和测试集利用时序数据构造新特征
x_train = get_t(df_train)
x_test = get_t(df_test)
print(x_train.head())
print(x_test.head())

       mean       std       var       min       max    median      skew  \
0  0.004765  0.583343  0.340289 -3.003019  3.236114  0.007919  0.000519   
1  0.015331  0.065739  0.004322 -0.222801  0.221132  0.017524 -0.205528   
2  0.003270  0.540327  0.291954 -5.501272  5.912233  0.002843  0.231480   
3  0.004026  0.366301  0.134176 -3.955295  3.712455  0.003249 -0.492159   
4  0.013258  0.233977  0.054745 -1.754299  1.550606  0.010802 -0.048808   

        kuri  
0   4.984610  
1   0.036342  
2  21.957896  
3  34.973034  
4   5.036917  
       mean       std       var       min       max    median      skew  \
0  0.003489  0.537096  0.288472 -4.631026  5.202391  0.003249 -0.019100   
1  0.006061  0.809820  0.655808 -2.891751  3.168703 -0.003452  0.085543   
2  0.006581  0.361151  0.130430 -3.624334  3.813977  0.006497 -0.474609   
3  0.009584  0.063453  0.004026 -0.243246  0.197559  0.011891 -0.156539   
4  0.004482  0.152550  0.023271 -0.518818  0.556503  0.003411  0.046709   

        

In [30]:
def to_file(label, df_test):
    """
    将预测的label组合成csv文件输出
    """
    result = pd.DataFrame()
    result['id'] = df_test['id']
    result['label'] = label
    result.to_csv('data/result.csv', index=False, encoding="utf-8")

In [53]:
# 随机森林在验证集上表现不错
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
clf = RandomForestClassifier(n_estimators=50)
x_tra, x_val, y_tra, y_val = train_test_split(x_train, y_train.values.reshape(len(y_train), ), test_size=0.2)
clf.fit(x_tra, y_tra)
print(confusion_matrix(y_val, clf.predict(x_val)))
print(accuracy_score(y_val, clf.predict(x_val)))

[[38  0  0  0  0  0  0  0  0  0]
 [ 0 10  0  0  0  0  0  0  0  0]
 [ 0  0  9  0  0  0  0  0  1  0]
 [ 0  0  1  7  0  0  0  0  0  0]
 [ 0  0  0  0  8  0  0  0  0  0]
 [ 0  0  0  0  0 11  0  0  0  0]
 [ 0  0  0  0  0  0 11  0  0  0]
 [ 0  0  0  0  0  0  0 27  0  0]
 [ 0  0  0  0  0  0  0  0  8  0]
 [ 0  0  0  0  0  0  0  0  0 28]]
0.9874213836477987


In [43]:
rst = clf.predict(x_test)
to_file(rst, df_test)